The `concurrent.futures` module
----

Concurrent processes are processes that will return the same results regardless of the order in which they were executed. A "future" is something that will return a result sometime in the future.  The `concurrent.futures` module provides an event handler, which can be fed functions to be scheduled for future execution. This provides us with a simple model for parallel execution on a multi-core machine.

While concurrent futures provide a simpler interface, it is slower and less flexible when compared with using `multiprocessing` for parallel execution.

In [1]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [3]:
import time

### Functions that the concurrent workers witll execute

In [2]:
def f1(a):
    return a

def f2(a, b):
    time.sleep(np.random.uniform(1, 3))
    return a + b

### Using ThreadPoolExecutor

In [4]:
with ProcessPoolExecutor(max_workers=4) as pool:
    a = pool.submit(f2, 1, 1)
    b = pool.submit(f2, 1,2)
    c = pool.submit(f1, 10)    

    print('a running:', a.running())
    print('a done:', a.done())

    print('b running:', b.running())
    print('b done:', b.done())

    print('c running:', c.running())
    print('c done:', c.done())

    print('a result', a.result())
    print('b result', b.result())
    print('c result', c.result())

a running: True
a done: False
b running: False
b done: False
c running: False
c done: False
a result 2
b result 3
c result 10


### Cancelling jobs and adding callbacks

In [109]:
njobs = 24

res = []

with ProcessPoolExecutor(max_workers=4) as pool:

    for i in range(njobs):
        res.append(pool.submit(f2, *np.random.rand(2)))
        if i % 2 == 0:
            res[i].add_done_callback(lambda future: print("Process done!"))
    res[4].cancel()
    if res[4].cancelled():
        print("Process 4 cancelled")

    for i, x in enumerate(res):
        while x.running():
            print("Running")
            time.sleep(1)
        if not x.cancelled():
            print(x.result())

Process done!
Process 4 cancelled
Running
Running
Process done!
1.02122163695
Running
Process done!
0.818612111779
0.630156341875
0.554480838967
Running
Running
Process done!
1.08430234507
1.26300606548
1.40121410958
Running
Process done!
Process done!
1.59545883957
Running
Running
0.880369459559
0.765422161684
1.01003602635
Running
Process done!
Process done!
1.57588563093
1.60058840092
1.52164884486
Running
Process done!
1.66773104245
Running
Process done!
Process done!
1.20410753922
0.864620207314
0.149761417977
Running
1.41591996706
1.2745396651
Running
Process done!
0.27440516728
0.613841815446
Running
0.926824352048


### Using ProcessPoolExecutor with map

Unlike threads, processes are not subject to the Global Interprer Lock and should be used for parllel tasks that are CPU-bound rather than I/O-bound. The interface for ProcessPoolExecutor is identitcal to that of ThreadPoolExecutor.

In [101]:
xs = np.ones(24)

#### Trick to use map with function that expects multiple arguments

In [ ]:
def f2_(args):
    return f2(*args)

In [81]:
with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(f2_, np.array_split(xs, xs.shape[0]//2))
list(res)

[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]